In [2]:
%matplotlib inline
import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt
import os,sys
from PIL import Image
import re
import cv2

from keras.layers import Dense, Dropout, Flatten, Reshape, Conv2D, MaxPooling2D, LeakyReLU
from keras import backend as K
from keras.optimizers import Adam

from keras.models import model_from_json

from helpers import *

Using TensorFlow backend.


In [3]:
# helper variables and functions for the submission
foreground_threshold = 0.5 # percentage of pixels > 1 required to assign a foreground label to a patch
patch_size = 16

def pred_print(img, img_patches):
    pred = np.round(model.predict(img_patches, verbose=1))
    disp_img_pred(img, pred)
    return pred


# Creates submission file using a trained model and given images
def submission_with_model(model, submissionfilename):

    root_dir = "data/test_set_images/"

    # Get filenames and images for all the 50 submission images
    image_dir = [root_dir + "test_{}/".format(i) for i in range(1, 51)]
    filenames = [fn for imdir in image_dir for fn in os.listdir(imdir)]
    images = [load_image(image_dir[i-1] + filenames[i-1]) for i in range(1, 51)]

    # Reflect borders
    im_borders = [reflect_border(im) for im in images]
    
    # Get 16x16 patches of the images
    imgs_patched = [image_to_inputs(im, patch_size) for im in im_borders]
    
    # Use model to predict, predictions are labels in a list of labels per image
    predictions = []
    for i in range(1,51):
        predictions.append(pred_print(images[i-1], np.asarray(imgs_patched[i-1])))
    
    with open(submissionfilename, 'w') as f:
        f.write('id,prediction\n')
        for nr in range(1, 51):
            f.writelines(
                '{}\n'.format(s)
                for s in img_to_submission_strings(predictions[nr-1], nr, images[nr-1].shape[1], images[nr-1].shape[0])
            )

    

In [4]:
# load the model from data
path = 'savedModels/'
fm = open(path + 'model_18')
model = model_from_json(fm.read())
fm.close()

model.load_weights(path + 'weights_18', by_name=False)

model.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=1e-3),#'rmsprop',
              metrics=['accuracy'])


In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 77, 77, 16)        784       
_________________________________________________________________
leaky_re_lu_13 (LeakyReLU)   (None, 77, 77, 16)        0         
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 38, 38, 16)        0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 38, 38, 16)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 35, 35, 32)        8224      
_________________________________________________________________
leaky_re_lu_14 (LeakyReLU)   (None, 35, 35, 32)        0         
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 17, 17, 32)        0         
__________

In [6]:
# Use the model to predict and create the submission file.
submission_with_model(model, 'submission_test.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'data/test_set_images/test_2/.directory'